In [76]:
# !pip install altair 
# !pip install altair_saver
# !npm install -g vega-lite vega-cli canvas
# !pip install vl-convert-python

In [77]:
# !apt update
# !apt install ttf-mscorefonts-installer -y
# !apt reinstall fontconfig fontconfig-config libfontconfig1 -y

In [78]:
# !wget https://raw.githubusercontent.com/EconomicsObservatory/ECOvisualisations/main/guidelines/fonts/Circular/CircularStd-Black.otf -P /usr/local/share/fonts
# !wget https://raw.githubusercontent.com/EconomicsObservatory/ECOvisualisations/main/guidelines/fonts/Circular/CircularStd-Bold.otf -P /usr/local/share/fonts
# !wget https://raw.githubusercontent.com/EconomicsObservatory/ECOvisualisations/main/guidelines/fonts/Circular/CircularStd-Book.otf -P /usr/local/share/fonts
# !wget https://raw.githubusercontent.com/EconomicsObservatory/ECOvisualisations/main/guidelines/fonts/Circular/CircularStd-Medium.otf -P /usr/local/share/fonts

In [79]:
# !fc-cache -f

In [80]:
import json
import altair as alt
from altair import expr, datum
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import requests
import urllib.parse

In [81]:
import colorsys
from matplotlib.colors import to_hex, to_rgb


def scale_lightness(rgb, scale_l):
    rgbhex = False
    if "#" in rgb:
        rgb = to_rgb(rgb)
        rgbhex = True
    # convert rgb to hls
    h, l, s = colorsys.rgb_to_hls(*rgb)
    # manipulate h, l, s values and return as rgb
    c = colorsys.hls_to_rgb(h, min(1, l * scale_l), s=s)
    if rgbhex:
        c = to_hex(c)
    return c

In [536]:
SAVE = True
LOCAL = False
DARK = True

if LOCAL:
    local_suffix = "" #_local
else:
    local_suffix = ""

In [537]:
%%capture pwd
!pwd

In [516]:
uid = pwd.stdout.split("/")[-1].split("\r")[0]
uid=urllib.parse.quote(uid)
eco_git_home = (
    "https://raw.githubusercontent.com/EconomicsObservatory/ECOvisualisations/main/"
)
eco_git_path = eco_git_home + "magazine/" + uid + "/data/"
vega_embed = requests.get(eco_git_home + "guidelines/html/vega-embed.html").text
colors = json.loads(
    requests.get(eco_git_home + "guidelines/colors/eco-colors.json").content
)
category_color = json.loads(
    requests.get(eco_git_home + "guidelines/colors/eco-category-color.json").content
)
hue_color = json.loads(
    requests.get(eco_git_home + "guidelines/colors/eco-single-hue-color.json").content
)
mhue_color = json.loads(
    requests.get(eco_git_home + "guidelines/colors/eco-multi-hue-color.json").content
)
div_color = json.loads(
    requests.get(eco_git_home + "guidelines/colors/eco-diverging-color.json").content
)
config = json.loads(
    requests.get(eco_git_home + "guidelines/charts/eco-global-config.json").content
)
mo=0.5
mo0=0.01
height = config["height"]
width = config["width"]
uid, height, width

('what-do-the-latest-data-tell-us-about-uk-wage-growth', 300, 500)

In [517]:
def save(df, f, LOCAL):
    fc = eco_git_path + f + ".csv"
    df.to_csv("data/" + f + ".csv")
    f += local_suffix
    open("visualisation/" + f + ".html", "w").write(
        vega_embed.replace(
            "JSON_PATH", fc.replace("/data/", "/visualisation/").replace(".csv", ".json")
        )
    )
    if LOCAL:
        fc = df
    
    from IPython.display import display, HTML
    display(HTML(df.head().to_html()))
    
    # readme = "## Figure " + f.replace('fig','').split('_')[0] + \
    
    readme = "## Figure " + f.replace('fig','').split('_')[0] + \
        '  \n\nData: [`csv`](data/' + f + '.csv)' +\
        '  \nGitHub: [' + f + '](https://github.com/EconomicsObservatory/ECOvisualisations/tree/main/magazine/'+uid +'#figure-'+f.split('_')[0][3:]+')'+\
        ''+\
        '  \n\n### Light theme  \n\nVersions with data locally embedded into the `Vega-lite` specification file: ' + \
        '[`png`](visualisation/' + f + '_local.png) [`svg`](visualisation/' + f + local_suffix +'.svg) [`json`](visualisation/' + f + local_suffix +'.json) '+ \
        ''+\
        '  \n\n### Dark theme  \n\nVersions with data locally embedded into the `Vega-lite` specification file: ' + \
        '[`png`](visualisation/' + f + '_local_dark.png) [`svg`](visualisation/' + f + local_suffix +'_dark.svg) [`json`](visualisation/' + f + local_suffix+ '_dark.json) '+ \
        ''+\
        '  \n\n!["' + f + '"](visualisation/' + f + '.svg "' + f + '")\n\n' +\
        '  \n\n!["' + f + '_dark"](visualisation/' + f + '_dark.svg "' + f + '")\n\n' 
    return readme, f, fc

In [518]:
def area(base,color,base_color='#ffffff00',opacity=1,kind='area'):
    if kind=='area':
        return base.mark_area(opacity=opacity,
        interpolate="monotone",
        line={'color':color},
        color=alt.Gradient(
            gradient='linear',
            stops=[alt.GradientStop(color=base_color, offset=0.2),
                   alt.GradientStop(color=color, offset=0.8)],
            x1=1, #0.8
            y1=1,
            x2=1,
            y2=0
            )
        )
    elif kind=='line':
        return base.mark_line(opacity=opacity,interpolate="monotone",color=color)
    elif kind=='dashed-line':
        return base.mark_line(opacity=opacity,interpolate="monotone",color=color,strokeDash=[5,5])
    else:
        print('Please specify kind...')
        return

In [519]:
# service_color='#d6c8da' '#e4bfe2' '#ce4b96' colors['eco-turquiose']
service_color='#b4c8d8'

def dark(f):
    # service_color='#d6c8da' '#e4bfe2' '#ce4b96' colors['eco-turquiose']
    service_color='#b4c8d8'
    configSource = "visualisation/" + f + ".json"
    config = json.loads(open(configSource, "r").read())
    config['background']=colors['eco-background']
    def dark_layer(config):
        def dark_color(color):
            return color\
                .replace(colors['eco-gray'],service_color)\
                .replace(colors['eco-blue'],colors['eco-yellow'])\
                .replace(colors['eco-dark-blue'],colors['eco-light-blue'])\
                .replace(colors['eco-purple'],'#FF69B4')\
                .replace('#eeeeee','#224183')\
                .replace('#cccddd','#1c376f')

        for i in config['layer']:
            if 'encoding' in i:
                for x in ['x','y','color']:
                    if x in i['encoding']:
                        if 'axis' in i['encoding'][x]:
                            for c in ['domainColor','labelColor','tickColor','titleColor','gridColor']:
                                if 'axis' in i['encoding'][x]:
                                    if i['encoding'][x]['axis']!=None:
                                        if c in i['encoding'][x]['axis']:
                                            i['encoding'][x]['axis'][c]=service_color
                        if 'scale' in i['encoding'][x]:
                            if 'range' in i['encoding'][x]['scale']:
                                i['encoding'][x]['scale']['range']=[dark_color(r) for r in i['encoding'][x]['scale']['range']]
                        if 'legend' in i['encoding'][x]:
                            if i['encoding'][x]['legend']!=None:
                                if 'labelColor' in i['encoding'][x]['legend']:
                                    i['encoding'][x]['legend']['labelColor']=dark_color(i['encoding'][x]['legend']['labelColor'])
                                if 'titleColor' in i['encoding'][x]['legend']:
                                    i['encoding'][x]['legend']['titleColor']=dark_color(i['encoding'][x]['legend']['titleColor'])
                            
            if 'mark' in i:
                if 'color' in i['mark']:
                    # manual dark_color overrides for marks only
                    if i['mark']['color']==service_color:
                        i['mark']['color']=colors['eco-green']
                    elif 'stops' in i['mark']['color']:
                        for s in i['mark']['color']['stops']:
                            if 'color' in s:
                                s['color']=dark_color(s['color'])
                    else:
                        i['mark']['color']=dark_color(i['mark']['color'])
                if 'line' in i['mark']:
                    if type(i['mark'])==str:
                        if 'color' in i['encoding']:
                            if 'scale' in i['encoding']['color']:
                                if 'range' in i['encoding']['color']['scale']:
                                    for color in i['encoding']['color']['scale']['range']:
                                        color=dark_color(color)
                    else:
                        if 'color' in i['mark']['line']:
                            i['mark']['line']['color']=dark_color(i['mark']['line']['color'])
                if 'fill' in i['mark']:
                    i['mark']['fill']=dark_color(i['mark']['fill'])
                if 'stroke' in i['mark']:
                    i['mark']['stroke']=dark_color(i['mark']['stroke'])
                    
        if 'title' in config:
            if 'color' in config['title']:
                    config['title']['color']=dark_color(config['title']['color'])
            if 'subtitle' in config['title']:
                if 'subtitleColor' in config['title']:
                    config['title']['subtitleColor']=dark_color(config['title']['subtitleColor'])
                    
        return config

    if 'vconcat' in config:
        for layer in config['vconcat']:
            layer=dark_layer(layer)
    elif 'hconcat' in config:
        for layer in config['hconcat']:
            layer=dark_layer(layer)
    else:
        config=dark_layer(config)
    
    if 'datasets' in config:
        for i in config['datasets']:
            if len(config['datasets'][i])>0:
                if 'img' in config['datasets'][i][0]:
                    if 'eco-icon-dark' in config['datasets'][i][0]['img']:
                        config['datasets'][i][0]['img']=config['datasets'][i][0]['img'].replace('eco-icon-dark','eco-icon-light')
                    if 'shipping-dark' in config['datasets'][i][0]['img']:
                        config['datasets'][i][0]['img']=config['datasets'][i][0]['img'].replace('shipping-dark','shipping-light')
    
    return alt.Chart.from_dict(config) 

# Global

In [520]:
rc={'North East': colors["eco-mid-blue"],
 'North West': colors["eco-dark-blue"],
 'Yorkshire and The Humber': colors["eco-turquiose"],
 'East Midlands': colors["eco-red"],
 'West Midlands': colors["eco-blue"],
 'East': colors["eco-gray"],
 'London': colors["eco-pink"],
 'South East': colors["eco-yellow"],
 'South West': colors["eco-orange"],
 'Wales': colors["eco-dot"],
 'Scotland': colors["eco-light-blue"],
 'Northern Ireland': colors["eco-green"],
 'UK': colors["eco-purple"]}

## Fig 1

In [521]:
df = pd.read_excel("raw/Chart1 (1).xls",sheet_name='data',skiprows=6)[['Unnamed: 0','Total pay ','Regular pay']]
df.columns=['Date','Total pay','Regular pay']
df['date2']=pd.to_datetime(df['Date'].str.split(' to ').str[-1])-pd.to_timedelta('30d')
# df['date']=pd.to_datetime((df.date.astype(str).str[:5]+(df.date.dt.month-1).astype(str)+df.date.astype(str).str[-3:]).str.replace('-0-','-12-'))

In [522]:
readme, f, fc = save(df,"fig1_wage",LOCAL)

,Date,Total pay,Regular pay,date2
0,Jan to Mar 2001,4.9,2.7,2001-01-30
1,Feb to Apr 2001,4.9,3.2,2001-03-02
2,Mar to May 2001,3.6,3.3,2001-04-01
3,Apr to Jun 2001,3.7,3.2,2001-05-02
4,May to Jul 2001,3.3,3.2,2001-06-01


In [523]:
base=alt.Chart(fc).transform_calculate(tp='datum["Total pay"]/100',rp='datum["Regular pay"]/100')
line1 = base.mark_line(opacity=0.8,clip=True,interpolate='monotone',color=colors['eco-light-blue']).encode(
    x=alt.X('date2:T',axis=alt.Axis(
            grid=True,
            gridDash=[1,5],
            gridColor=colors["eco-gray"],
            gridOpacity=mo,
            titleAlign="left",
            titleAnchor="end",
            title="",
        titleX=405,
            titleY=6,
            labelColor=colors["eco-gray"],
            titleColor=colors["eco-gray"],
            tickColor=colors["eco-gray"],
            domainColor=colors["eco-gray"],
            tickCount=10,
            domainOpacity=mo,
            tickOpacity=mo,
            labelOpacity=mo+0.2,
            titleOpacity=mo+0.3,
            titleFontSize=10,
            titleFontWeight='normal',
            orient="bottom",
            labelAngle=0
            ),
            scale=alt.Scale(domain=['2000-01-01','2024-01-01'],nice=False)
    ),
    y=alt.Y('tp:Q',axis=alt.Axis( 
            # grid=False,
            gridDash=[1,5],
            gridColor=colors["eco-gray"],
            gridOpacity=mo,
            title="Annual growth rate of average weekly earnings (%)",
            titleX=0,
            titleY=-7,
            titleBaseline="bottom",
            titleAngle=0,
            titleAlign="left",
            ticks=False,
            labelPadding=5,
            labelColor=colors["eco-gray"],
            titleColor=colors["eco-gray"],
            tickColor=colors["eco-gray"],
            domainColor=colors["eco-gray"],
            domainOpacity=mo,
            tickOpacity=mo,
            labelOpacity=mo+0.2,
            titleOpacity=mo+0.3,
            titleFontSize=12,
            titleFontWeight='normal',
            format='%',
            tickCount=8,
            # domain=False,
            # labels=False
        ),
            scale=alt.Scale(domain=[-0.08,0.08])
    
           )
)
line2=line1.mark_line(opacity=0.8,clip=True,interpolate='monotone',color=colors['eco-dot']).encode(y='rp:Q')
label1=line1.mark_text(align='left',dx=5,dy=4,color=colors['eco-light-blue']).encode(text='t:N').transform_filter('datum.date2>toDate("2023-03-10")').transform_calculate(t='"Total pay"'
)
label2=line2.mark_text(align='left',dx=5,dy=-4,color=colors['eco-dot']).encode(text='t:N').transform_filter('datum.date2>toDate("2023-03-10")').transform_calculate(t='"Regular pay"'
)
ruler=alt.Chart(pd.DataFrame([{'y':0}])).mark_rule(color=colors['eco-gray'],opacity=0.3,size=2).encode(y='y:Q')
area=base.mark_area(opacity=0.3,
        interpolate="monotone",
        line={'color':"#ffffff"},
        color=alt.Gradient(
            gradient='linear',
            stops=[alt.GradientStop(color=colors['eco-dot'], offset=0.2),
                   alt.GradientStop(color=colors['eco-light-blue'], offset=0.8)],
            x1=1, #0.8
            y1=1,
            x2=1,
            y2=0
            )
).encode(x='date2:T',y='tp:Q',y2='rp:Q')
tt=area.mark_line(opacity=0.01,size=10,interpolate='monotone').encode(tooltip=[
             alt.Tooltip('date:T',title='Date',format='%B %Y'),
             alt.Tooltip('tp:Q',title='Total pay growth rate (%)',format='.2%'),
            alt.Tooltip('rp:Q',title='Regular pay growth rate (%)',format='.2%')])

layer1=(ruler+area+line1+line2+label1+label2+tt
       ).\
    configure(font='Circular Std Book').configure_view(stroke=None,discreteWidth=400,discreteHeight=300,continuousWidth=400,continuousHeight=300).properties( 
        title={'text':"Inflation-adjusted growth in average weekly earnings",'color':colors['eco-gray']+'dd','align':'left','anchor':'start','offset':-360,
              'subtitle':'Source: ONS, 2023','subtitlePadding':360,
               'subtitleFontSize':11,'subtitleColor':colors['eco-gray']+'bb','fontSize':15})

if SAVE:
    layer1.save("visualisation/" + f + ".json")
    layer1.save("visualisation/" + f + ".svg")
    layer1.save("visualisation/" + f + ".png")
    layer2=dark(f)
    layer2.save("visualisation/" + f + "_dark.json")
    layer2.save("visualisation/" + f + "_dark.svg")
    layer2.save("visualisation/" + f + "_dark.png")
    open("README.md", "w").write(readme)

print(f+'\n')
layer1.display()
if SAVE:
    layer2.display()
    
# force display dark
if DARK and not SAVE:
    layer1.save("visualisation/" + f + ".json")
    layer2=dark(f)
    layer2.display()

WARN Line mark is for continuous lines and thus cannot be used with y2. We will use the rule mark (line segments) instead.
WARN Line mark is for continuous lines and thus cannot be used with y2. We will use the rule mark (line segments) instead.
WARN Line mark is for continuous lines and thus cannot be used with y2. We will use the rule mark (line segments) instead.
WARN Line mark is for continuous lines and thus cannot be used with y2. We will use the rule mark (line segments) instead.


fig1_wage



alt.LayerChart(...)

alt.LayerChart(...)

## Fig 2

In [524]:
df=pd.read_excel('raw/Charts2 (1).xlsx',sheet_name='9. Mean pay (NUTS1)',usecols='P:AC',skiprows=18).set_index('Unnamed: 15').stack().reset_index()
df.columns=['date','region','value']
df['date']=pd.to_datetime(df['date'])

In [525]:
readme, f, fc = save(df,"fig2_wage-by-region",LOCAL)

,date,region,value
0,2015-07-01,North East,0.005238
1,2015-07-01,North West,0.010309
2,2015-07-01,Yorkshire and The Humber,0.017039
3,2015-07-01,East Midlands,0.015003
4,2015-07-01,West Midlands,0.018789


In [526]:
base=alt.Chart(fc)
lines = base.mark_line(opacity=0.4,clip=True,interpolate='monotone').encode(
    x=alt.X('date:T',axis=alt.Axis(
            grid=True,
            gridDash=[1,5],
            gridColor=colors["eco-gray"],
            gridOpacity=mo,
            titleAlign="left",
            titleAnchor="end",
            title="",
        titleX=405,
            titleY=6,
            labelColor=colors["eco-gray"],
            titleColor=colors["eco-gray"],
            tickColor=colors["eco-gray"],
            domainColor=colors["eco-gray"],
            tickCount=10,
            domainOpacity=mo,
            tickOpacity=mo,
            labelOpacity=mo+0.2,
            titleOpacity=mo+0.3,
            titleFontSize=10,
            titleFontWeight='normal',
            orient="bottom",
            labelAngle=0,
            # scale=alt.Scale(domain=[1700,2100])
    )),
    y=alt.Y('value:Q',axis=alt.Axis( 
            # grid=False,
            gridDash=[1,5],
            gridColor=colors["eco-gray"],
            gridOpacity=mo,
            title="Annual growth rate of average pay (%)",
            titleX=0,
            titleY=-7,
            titleBaseline="bottom",
            titleAngle=0,
            titleAlign="left",
            ticks=False,
            labelPadding=5,
            labelColor=colors["eco-gray"],
            titleColor=colors["eco-gray"],
            tickColor=colors["eco-gray"],
            domainColor=colors["eco-gray"],
            domainOpacity=mo,
            tickOpacity=mo,
            labelOpacity=mo+0.2,
            titleOpacity=mo+0.3,
            titleFontSize=12,
            titleFontWeight='normal',
            format='%',
            tickCount=8,
            # domain=False,
            # labels=False
        ),
            # scale=alt.Scale(type='log',domain=[400,200000])
    
           ),
    color=alt.Color('region:N',sort=[],
                    legend=alt.Legend(title='Region',
                                                 titleColor=colors["eco-gray"],
                                                 labelColor=colors["eco-gray"]),scale=alt.Scale(
        range=list(rc.values()),domain=list(rc.keys())
    )),
)
uk=lines.mark_line(opacity=0.8,clip=True,interpolate='monotone',color=colors['eco-purple'],strokeWidth=3).transform_filter('datum.region=="UK"')
tt=lines.mark_line(opacity=0.01,size=10,interpolate='monotone').encode(tooltip=[alt.Tooltip('region:N',title='Region'),
             alt.Tooltip('date:T',title='Date',format='%B %Y'),
             alt.Tooltip('value:Q',title='Growth rate (%)',format='.2%')])

labels=lines.mark_text(opacity=0.8,align='left',dx=5).encode(text='t:N').transform_filter('datum.date>toDate("2023-04-01")').transform_calculate(t='datum.region'
).transform_filter('datum.region=="UKX"')

layer1=(lines+uk+labels+tt).\
    configure(font='Circular Std Book').configure_view(stroke=None,discreteWidth=400,discreteHeight=300,continuousWidth=400,continuousHeight=300).properties( 
        title={'text':"Annual growth rate of average pay by region and nation of the UK",'color':colors['eco-gray']+'dd','align':'left','anchor':'start','offset':-360,
              'subtitle':'Source: ONS, 2023','subtitlePadding':360,
               'subtitleFontSize':11,'subtitleColor':colors['eco-gray']+'bb','fontSize':15})

if SAVE:
    layer1.save("visualisation/" + f + ".json")
    layer1.save("visualisation/" + f + ".svg")
    layer1.save("visualisation/" + f + ".png")
    layer2=dark(f)
    layer2.save("visualisation/" + f + "_dark.json")
    layer2.save("visualisation/" + f + "_dark.svg")
    layer2.save("visualisation/" + f + "_dark.png")
    open("README.md", "a").write(readme)

print(f+'\n')
layer1.display()
if SAVE:
    layer2.display()
    
# force display dark
if DARK and not SAVE:
    layer1.save("visualisation/" + f + ".json")
    layer2=dark(f)
    layer2.display()

WARN Infinite extent for field "date": [Infinity, -Infinity]
WARN Infinite extent for field "value": [Infinity, -Infinity]
WARN Infinite extent for field "date": [Infinity, -Infinity]
WARN Infinite extent for field "value": [Infinity, -Infinity]
WARN Infinite extent for field "date": [Infinity, -Infinity]
WARN Infinite extent for field "value": [Infinity, -Infinity]
WARN Infinite extent for field "date": [Infinity, -Infinity]
WARN Infinite extent for field "value": [Infinity, -Infinity]


fig2_wage-by-region



alt.LayerChart(...)

alt.LayerChart(...)

## Fig 3

In [527]:
df=pd.read_excel('raw/Charts2 (1).xlsx',sheet_name='9. Mean pay (NUTS1)',usecols='A:N',skiprows=117,nrows=107).set_index('Unnamed: 0').stack().reset_index()
df.columns=['date','region','value']
df['date']=pd.to_datetime(df['date'])

In [528]:
readme, f, fc = save(df,"fig3-median-pay",LOCAL)

,date,region,value
0,2014-07-01,North East,0.798324
1,2014-07-01,North West,0.779381
2,2014-07-01,Yorkshire and The Humber,0.794462
3,2014-07-01,East Midlands,0.781169
4,2014-07-01,West Midlands,0.792276


In [529]:
base=alt.Chart(fc)
lines = base.mark_line(opacity=0.8,clip=True,interpolate='monotone').encode(
    x=alt.X('date:T',axis=alt.Axis(
            grid=True,
            gridDash=[1,5],
            gridColor=colors["eco-gray"],
            gridOpacity=mo,
            titleAlign="left",
            titleAnchor="end",
            title="",
        titleX=405,
            titleY=6,
            labelColor=colors["eco-gray"],
            titleColor=colors["eco-gray"],
            tickColor=colors["eco-gray"],
            domainColor=colors["eco-gray"],
            tickCount=10,
            domainOpacity=mo,
            tickOpacity=mo,
            labelOpacity=mo+0.2,
            titleOpacity=mo+0.3,
            titleFontSize=10,
            titleFontWeight='normal',
            orient="bottom",
            labelAngle=0,
            # scale=alt.Scale(domain=[1700,2100])
    )),
    y=alt.Y('value:Q',axis=alt.Axis( 
            # grid=False,
            gridDash=[1,5],
            gridColor=colors["eco-gray"],
            gridOpacity=mo,
            title="Median pay as a % of average pay",
            titleX=0,
            titleY=-7,
            titleBaseline="bottom",
            titleAngle=0,
            titleAlign="left",
            ticks=False,
            labelPadding=5,
            labelColor=colors["eco-gray"],
            titleColor=colors["eco-gray"],
            tickColor=colors["eco-gray"],
            domainColor=colors["eco-gray"],
            domainOpacity=mo,
            tickOpacity=mo,
            labelOpacity=mo+0.2,
            titleOpacity=mo+0.3,
            titleFontSize=12,
            titleFontWeight='normal',
            format='%',
            tickCount=8,
            # domain=False,
            # labels=False
        ),
            scale=alt.Scale(domain=[.55,.9],nice=False)
    
           ),
    color=alt.Color('region:N',sort=[],
                    legend=None,scale=alt.Scale(
        range=list(rc.values()),domain=list(rc.keys())
    )),
)
uk=lines.mark_line(opacity=0.8,clip=True,interpolate='monotone',color=colors['eco-purple'],strokeWidth=3).transform_filter('datum.region=="UK"')
tt=lines.mark_line(opacity=0.01,size=10,interpolate='monotone').encode(tooltip=[alt.Tooltip('region:N',title='Region'),
             alt.Tooltip('date:T',title='Date',format='%B %Y'),
             alt.Tooltip('value:N',title='Median as % of average',format='.2%')])

labels=lines.mark_text(opacity=0.8,align='left',dx=5).encode(text='t:N').transform_filter('datum.date>toDate("2023-04-01")').transform_calculate(t='datum.region'
)
labels1=labels.transform_filter({'not': alt.FieldOneOfPredicate(oneOf=['South West',
 'Northern Ireland','Wales','East Midlands','West Midlands','North West','Yorkshire and The Humber'],field="region")})
labels2=labels.mark_text(align='left',dx=5,dy=27).transform_filter('datum.region=="South West"')
labels3=labels.mark_text(align='left',dx=5,dy=24).transform_filter('datum.region=="East Midlands"')
labels4=labels.mark_text(align='left',dx=5,dy=12).transform_filter('datum.region=="West Midlands"')
labels5=labels.mark_text(align='left',dx=55,dy=0).transform_filter('datum.region=="North West"')
labels6=labels.mark_text(align='left',dx=5,dy=-15).transform_filter('datum.region=="Northern Ireland"')
labels7=labels.mark_text(align='left',dx=5,dy=-5).transform_filter('datum.region=="Wales"')
labels8=labels.mark_text(align='left',dx=5,dy=3).transform_filter('datum.region=="Yorkshire and The Humber"')

layer1=(lines+uk+labels1+labels2+labels3+labels4+labels5+labels6+labels7+labels8+tt).\
    configure(font='Circular Std Book').configure_view(stroke=None,discreteWidth=400,discreteHeight=300,continuousWidth=400,continuousHeight=300).properties( 
        title={'text':"Median pay as a percentage of average pay",'color':colors['eco-gray']+'dd','align':'left','anchor':'start','offset':-360,
              'subtitle':'Source: ONS, 2023','subtitlePadding':360,
               'subtitleFontSize':11,'subtitleColor':colors['eco-gray']+'bb','fontSize':15})

if SAVE:
    layer1.save("visualisation/" + f + ".json")
    layer1.save("visualisation/" + f + ".svg")
    layer1.save("visualisation/" + f + ".png")
    layer2=dark(f)
    layer2.save("visualisation/" + f + "_dark.json")
    layer2.save("visualisation/" + f + "_dark.svg")
    layer2.save("visualisation/" + f + "_dark.png")
    open("README.md", "a").write(readme)

print(f+'\n')
layer1.display()
if SAVE:
    layer2.display()
    
# force display dark
if DARK and not SAVE:
    layer1.save("visualisation/" + f + ".json")
    layer2=dark(f)
    layer2.display()

fig3-median-pay



alt.LayerChart(...)

alt.LayerChart(...)

## Fig 4

In [530]:
df=pd.read_excel('raw/Copy of Charts2 (1).xlsx',sheet_name='9. Mean pay (NUTS1)',usecols='A:N',skiprows=234,nrows=107).set_index('Unnamed: 0').stack().reset_index()
df.columns=['date','region','value']
df['date']=pd.to_datetime(df['date'])

In [531]:
readme, f, fc = save(df,"fig4-ratio",LOCAL)

,date,region,value
0,2014-07-01,North East,1.0
1,2014-07-01,North West,1.0
2,2014-07-01,Yorkshire and The Humber,1.0
3,2014-07-01,East Midlands,1.0
4,2014-07-01,West Midlands,1.0


In [532]:
base=alt.Chart(fc)
lines = base.mark_line(opacity=0.8,clip=True,interpolate='monotone').encode(
    x=alt.X('date:T',axis=alt.Axis(
            grid=True,
            gridDash=[1,5],
            gridColor=colors["eco-gray"],
            gridOpacity=mo,
            titleAlign="left",
            titleAnchor="end",
            title="",
        titleX=405,
            titleY=6,
            labelColor=colors["eco-gray"],
            titleColor=colors["eco-gray"],
            tickColor=colors["eco-gray"],
            domainColor=colors["eco-gray"],
            tickCount=10,
            domainOpacity=mo,
            tickOpacity=mo,
            labelOpacity=mo+0.2,
            titleOpacity=mo+0.3,
            titleFontSize=10,
            titleFontWeight='normal',
            orient="bottom",
            labelAngle=0,
            # scale=alt.Scale(domain=[1700,2100])
    )),
    y=alt.Y('value:Q',axis=alt.Axis( 
            # grid=False,
            gridDash=[1,5],
            gridColor=colors["eco-gray"],
            gridOpacity=mo,
            title="Ratio of average to median pay, July 2014 = 1",
            titleX=0,
            titleY=-7,
            titleBaseline="bottom",
            titleAngle=0,
            titleAlign="left",
            ticks=False,
            labelPadding=5,
            labelColor=colors["eco-gray"],
            titleColor=colors["eco-gray"],
            tickColor=colors["eco-gray"],
            domainColor=colors["eco-gray"],
            domainOpacity=mo,
            tickOpacity=mo,
            labelOpacity=mo+0.2,
            titleOpacity=mo+0.3,
            titleFontSize=12,
            titleFontWeight='normal',
            format='.2f',
            tickCount=8,
            # domain=False,
            # labels=False
        ),
            scale=alt.Scale(domain=[.91,1.06],nice=False)
    
           ),
    color=alt.Color('region:N',
                    sort=[],
                    legend=alt.Legend(title='Region',
                                                 titleColor=colors["eco-gray"],
                                                 labelColor=colors["eco-gray"]),
                    scale=alt.Scale(
        range=list(rc.values()),domain=list(rc.keys())
    )),
)
uk=lines.mark_line(opacity=0.8,clip=True,interpolate='monotone',color=colors['eco-purple'],strokeWidth=3).transform_filter('datum.region=="UK"')
tt=lines.mark_line(opacity=0.01,size=10,interpolate='monotone').encode(tooltip=[alt.Tooltip('region:N',title='Region'),
             alt.Tooltip('date:T',title='Date',format='%B %Y'),
             alt.Tooltip('value:Q',title='Ratio',format='.2f')])

labels=lines.mark_text(opacity=0.8,align='left',dx=5).encode(text='t:N').transform_filter('datum.date>toDate("2023-04-01")').transform_calculate(t='datum.region'
)
labels1=labels.transform_filter({'not': alt.FieldOneOfPredicate(oneOf=['South West',
 'Northern Ireland','Wales','East Midlands','West Midlands','North West','Yorkshire and The Humber'],field="region")})
labels2=labels.mark_text(align='left',dx=5,dy=27).transform_filter('datum.region=="South West"')
labels3=labels.mark_text(align='left',dx=5,dy=24).transform_filter('datum.region=="East Midlands"')
labels4=labels.mark_text(align='left',dx=5,dy=12).transform_filter('datum.region=="West Midlands"')
labels5=labels.mark_text(align='left',dx=55,dy=0).transform_filter('datum.region=="North West"')
labels6=labels.mark_text(align='left',dx=5,dy=-15).transform_filter('datum.region=="Northern Ireland"')
labels7=labels.mark_text(align='left',dx=5,dy=-5).transform_filter('datum.region=="Wales"')
labels8=labels.mark_text(align='left',dx=5,dy=3).transform_filter('datum.region=="Yorkshire and The Humber"')

layer1=(lines+uk+tt).\
    configure(font='Circular Std Book').configure_view(stroke=None,discreteWidth=400,discreteHeight=300,continuousWidth=400,continuousHeight=300).properties( 
        title={'text':"Ratio of average to median pay, by region and nation of the UK",'color':colors['eco-gray']+'dd','align':'left','anchor':'start','offset':-360,
              'subtitle':'Source: ONS, 2022','subtitlePadding':360,
               'subtitleFontSize':11,'subtitleColor':colors['eco-gray']+'bb','fontSize':15})

if SAVE:
    layer1.save("visualisation/" + f + ".json")
    layer1.save("visualisation/" + f + ".svg")
    layer1.save("visualisation/" + f + ".png")
    layer2=dark(f)
    layer2.save("visualisation/" + f + "_dark.json")
    layer2.save("visualisation/" + f + "_dark.svg")
    layer2.save("visualisation/" + f + "_dark.png")
    open("README.md", "a").write(readme)

print(f+'\n')
layer1.display()
if SAVE:
    layer2.display()
    
# force display dark
if DARK and not SAVE:
    layer1.save("visualisation/" + f + ".json")
    layer2=dark(f)
    layer2.display()

fig4-ratio



alt.LayerChart(...)

alt.LayerChart(...)

## Fig 5

In [533]:
df=pd.read_excel('raw/Copy of Charts2 (1).xlsx',sheet_name='9. Mean pay (NUTS1)',usecols='A:N',skiprows=344,nrows=41).set_index('Unnamed: 0').stack().reset_index()
df.columns=['date','region','value']
df['date']=pd.to_datetime(df['date'])

In [534]:
readme, f, fc = save(df,"fig5_ratio2020",LOCAL)

,date,region,value
0,2020-01-01,North East,1.0
1,2020-01-01,North West,1.0
2,2020-01-01,Yorkshire and The Humber,1.0
3,2020-01-01,East Midlands,1.0
4,2020-01-01,West Midlands,1.0


In [535]:
base=alt.Chart(fc).transform_filter('datum.date>toDate("2019-12-31")')
lines = base.mark_line(opacity=0.8,clip=True,interpolate='monotone').encode(
    x=alt.X('date:T',axis=alt.Axis(
            grid=True,
            gridDash=[1,5],
            gridColor=colors["eco-gray"],
            gridOpacity=mo,
            titleAlign="left",
            titleAnchor="end",
            title="",
        titleX=405,
            titleY=6,
            labelColor=colors["eco-gray"],
            titleColor=colors["eco-gray"],
            tickColor=colors["eco-gray"],
            domainColor=colors["eco-gray"],
            tickCount=10,
            domainOpacity=mo,
            tickOpacity=mo,
            labelOpacity=mo+0.2,
            titleOpacity=mo+0.3,
            titleFontSize=10,
            titleFontWeight='normal',
            orient="bottom",
            labelAngle=0,
            # scale=alt.Scale(domain=[1700,2100])
    )),
    y=alt.Y('value:Q',axis=alt.Axis( 
            # grid=False,
            gridDash=[1,5],
            gridColor=colors["eco-gray"],
            gridOpacity=mo,
            title="Ratio of average to median pay, January 2020 = 1",
            titleX=0,
            titleY=-7,
            titleBaseline="bottom",
            titleAngle=0,
            titleAlign="left",
            ticks=False,
            labelPadding=5,
            labelColor=colors["eco-gray"],
            titleColor=colors["eco-gray"],
            tickColor=colors["eco-gray"],
            domainColor=colors["eco-gray"],
            domainOpacity=mo,
            tickOpacity=mo,
            labelOpacity=mo+0.2,
            titleOpacity=mo+0.3,
            titleFontSize=12,
            titleFontWeight='normal',
            format='.2f',
            tickCount=8,
            # domain=False,
            # labels=False
        ),
            scale=alt.Scale(domain=[.94,1.04],nice=False)
    
           ),
    color=alt.Color('region:N',
                    sort=[],
                    legend=alt.Legend(title='Region',
                                                 titleColor=colors["eco-gray"],
                                                 labelColor=colors["eco-gray"]),
                    scale=alt.Scale(
        range=list(rc.values()),domain=list(rc.keys())
    )),
)
uk=lines.mark_line(opacity=0.8,clip=True,interpolate='monotone',color=colors['eco-purple'],strokeWidth=3).transform_filter('datum.region=="UK"')
tt=lines.mark_line(opacity=0.01,size=10,interpolate='monotone').encode(tooltip=[alt.Tooltip('region:N',title='Region'),
             alt.Tooltip('date:T',title='Date',format='%B %Y'),
             alt.Tooltip('value:Q',title='Ratio',format='.2f')])

labels=lines.mark_text(opacity=0.8,align='left',dx=5).encode(text='t:N').transform_filter('datum.date>toDate("2023-04-01")').transform_calculate(t='datum.region'
)
labels1=labels.transform_filter({'not': alt.FieldOneOfPredicate(oneOf=['South West',
 'Northern Ireland','Wales','East Midlands','West Midlands','North West','Yorkshire and The Humber'],field="region")})
labels2=labels.mark_text(align='left',dx=5,dy=27).transform_filter('datum.region=="South West"')
labels3=labels.mark_text(align='left',dx=5,dy=24).transform_filter('datum.region=="East Midlands"')
labels4=labels.mark_text(align='left',dx=5,dy=12).transform_filter('datum.region=="West Midlands"')
labels5=labels.mark_text(align='left',dx=55,dy=0).transform_filter('datum.region=="North West"')
labels6=labels.mark_text(align='left',dx=5,dy=-15).transform_filter('datum.region=="Northern Ireland"')
labels7=labels.mark_text(align='left',dx=5,dy=-5).transform_filter('datum.region=="Wales"')
labels8=labels.mark_text(align='left',dx=5,dy=3).transform_filter('datum.region=="Yorkshire and The Humber"')

layer1=(lines+uk+tt).\
    configure(font='Circular Std Book').configure_view(stroke=None,discreteWidth=400,discreteHeight=300,continuousWidth=400,continuousHeight=300).properties( 
        title={'text':"Ratio of average to median pay, by region and nation of the UK, post-2020",'color':colors['eco-gray']+'dd','align':'left','anchor':'start','offset':-360,
              'subtitle':'Source: ONS, 2023','subtitlePadding':360,
               'subtitleFontSize':11,'subtitleColor':colors['eco-gray']+'bb','fontSize':15})

if SAVE:
    layer1.save("visualisation/" + f + ".json")
    layer1.save("visualisation/" + f + ".svg")
    layer1.save("visualisation/" + f + ".png")
    layer2=dark(f)
    layer2.save("visualisation/" + f + "_dark.json")
    layer2.save("visualisation/" + f + "_dark.svg")
    layer2.save("visualisation/" + f + "_dark.png")
    open("README.md", "a").write(readme)

print(f+'\n')
layer1.display()
if SAVE:
    layer2.display()
    
# force display dark
if DARK and not SAVE:
    layer1.save("visualisation/" + f + ".json")
    layer2=dark(f)
    layer2.display()

fig5_ratio2020



alt.LayerChart(...)

alt.LayerChart(...)